In [1]:
#include parent folder
import os, sys, inspect

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import time
import timeit
import numpy as np
import pytest
import jax
import jax.numpy as jnp
import functools
import scipy
import pandas as pd

from src.domain import Domain
from src.misc import *

In [22]:
def VLJ1(v, w):   
    r = jnp.linalg.norm(v-w) # euclidian distance between points
    return 4 * (4*pow(1 / r, 12) - 2*pow(1 / r, 6))

def VLJ2(v, w):
    r = jnp.linalg.norm(v-w) # euclidian distance between points
    r6 = pow(1 / r, 6)    # recycle exponents
    return (16*r6*r6 - 8*r6)

def VLJ3(r):
    return 4 * (4*pow(1 / r, 12) - 2*pow(1 / r, 6))

VLJ1_jit = jax.jit(VLJ1)
VLJ2_jit = jax.jit(VLJ2)

def Epot_loops_nojit(pos): 
    # primitive Loop   
    v = to3D(pos)
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ1(v[i], v[j])    # using jit actually shaved off 20% form runtime  
    return E  

def Epot_loops_VLJ1_jit(v): 
    # primitive Loop   
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ1_jit(v[i], v[j])  
    return E 


def Epot_loops_VLJ2_jit(v): 
    # primitive Loop   
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ2_jit(v[i], v[j])   
    return E   

def Epot_broadcasted(x):
    n = len(x)
    broadcasted = x[:, np.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)   # reshape such that we have a list of distance vectors
    dist = np.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    dist = dist[dist != 0]    # remove 0s
    E = 0
    for d in dist:
        E += VLJ3(d)

    return E /2

def Epot_cdist(pos):
    return np.triu(scipy.spatial.distance.cdist(pos, pos, metric=VLJ1)).sum()

def Epot_cdist_jnp(pos):
    return jnp.triu(scipy.spatial.distance.cdist(pos, pos, metric=VLJ1)).sum()

Epot_loops_jit = jax.jit(Epot_loops_VLJ1_jit)
Epot_loops2_jit = jax.jit(Epot_loops_VLJ2_jit)

In [23]:
def benchMark(domain, Epot_func, label="", n=5):
    result = Epot_func(domain.pos)  # call so that the jit compiler does not affect the benchmark

    timer = timeit.Timer(functools.partial(Epot_func, domain.pos))
    t = timer.timeit(n) / n # Rrepeat n times and take average
    """
    t = 0
    for i in range(n):
        start = time.time()
        Epot_func(domain.pos)
        end = time.time()
        t += (end - start)

    t = t /n
    """

    return pd.DataFrame(data={
        'time': [t], 
        'label': [label], 
        "result" : [result],
        "n_particles" : [len(domain.pos)],
        "size" : [domain.length]
        })

domain = Domain()
domain.fill(10,20,1)
data = pd.DataFrame()

In [24]:
# slow  ones
n = 1
data = data.append(benchMark(domain, Epot_loops_nojit, "primitive loop, no jit", n))
data = data.append(benchMark(domain, Epot_loops_VLJ1_jit, "primitive loop, jit", n))

In [35]:
# fast ones but long compile time
n = 20
data = data.append(benchMark(domain, Epot_loops_jit, "loop, all jitted", n))
data = data.append(benchMark(domain, Epot_loops2_jit, "loops, all jitted, recycled exonents", n))

In [25]:
# broadcasted
n = 10
data = data.append(benchMark(domain, Epot_broadcasted, "broadcasted", n))

TypeError: slice indices must be integers or None or have an __index__ method

In [26]:
# results
data = data.drop_duplicates(subset=["label"])
data = data.sort_values(by="time", ascending=False)
data

,time,label,result,n_particles,size
0,0.220923,"primitive loop, no jit",-0.001249408,10,20
0,0.009306,"primitive loop, jit",-0.001249408,10,20


In [29]:
x = np.array([
    [1,1,1],
    [2,2,2],
    [3,3,3],
    [4,4,4]
])

domain = Domain()
domain.fill(5, 1, 1)
x = domain.pos
n = len(x)
broadcasted = x[:, np.newaxis, :] - x
broadcasted = broadcasted.reshape(n*n,3)
l = np.linalg.norm(broadcasted, axis=1)
l[l != 0]

array([0.73708982, 0.55480865, 0.27753123, 0.68815769, 0.73708982,
       0.9501852 , 0.69886296, 1.21739952, 0.55480865, 0.9501852 ,
       0.55095957, 0.5063818 , 0.27753123, 0.69886296, 0.55095957,
       0.86053834, 0.68815769, 1.21739952, 0.5063818 , 0.86053834])

In [52]:
def Epot_broadcasted(v):
    broadcasted = x[:, np.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)    # reshape such that we have a list of distance vectors
    dist = np.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    dist = dist[l != 0]    # remove 0s
    E = 0
    for d in dist:
        E += VLJ3(d)

    return E 

domain = Domain()
domain.fill(5, 1, 1)
x = domain.pos
Epot_broadcasted(x)

ValueError: cannot reshape array of size 75 into shape (100,3)